<h1>Creation of phenopackets from tabular data (individuals in columns)</h1>
<p>We will process <a href="https://pubmed.ncbi.nlm.nih.gov/17470496/" target="__blank">Gribaa, et al. (2007) A new form of childhood onset, autosomal recessive spinocerebellar ataxia and epilepsy is localized at 16q21-q23</a></p>
<p>pyphetools provides a convenient way of extracting HPO terms from typical tables presented in supplemental material. Typical tables can have the individuals in columns or rows. In this case, we extract data from TABLE.</p>
<p>This note shows how to work through the table and set up the pyphetools encoder. The table was copied from table 1 in the original publication.</p>
<p>Users can work on one column at a time and then generate a collection of <a href="https://pubmed.ncbi.nlm.nih.gov/35705716/" target="__blank">GA4GH phenopackets</a> to represent each patient included in the original supplemental material. These phenopackets can then be used for a variety of downstream applications.</p>

In [1]:
import phenopackets as php
from google.protobuf.json_format import MessageToDict, MessageToJson
from google.protobuf.json_format import Parse, ParseDict
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
import os
import sys
import numpy as np
from pyphetools.creation import *
from pyphetools.creation.simple_column_mapper import try_mapping_columns
# last tested with pyphetools 0.4.5

<h2>Importing HPO data</h2>
<p>pyphetools uses the Human Phenotype Ontology (HPO) to encode phenotypic features. The recommended way of doing this is to ingest the hp.json file using HpoParser, which in turn creates an HpoConceptRecognizer object. </p>
<p>The HpoParser can accept a hpo_json_file argument if you want to use a specific file. If the argument is not passed, it will download the latext hp.json file from the HPO GitHub site and store it in a new subdirectory called hpo_data. It will not download the file if the file is already downloaded.</p>

In [2]:
parser = HpoParser()
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
metadata = MetaData(created_by="ORCID:0000-0002-5648-2155")
metadata.default_versions_with_hpo(version=hpo_version)

<h2>Importing the supplemental table</h2>
<p>Here, we use the pandas library to import this file (note that the Python package called openpyxl must be installed to read Excel files with pandas, although the library does not need to be imported in this notebook). pyphetools expects a pandas DataFrame as input, and users can choose any input format available for pandas include CSV, TSV, and Excel, or can use any other method to transform their input data into a Pandas DataFrame before using pyphetools.</p>

In [3]:
df = pd.read_excel('input/PMID_17470496.xlsx')

In [4]:
df

,Patient,II,II2,II3,II4
0,Sex,female,female,male,female
1,Age,19,18,16,10
2,Seizures,+,+,+,+
3,Motor delay,+,+,+,+
4,Developmental delay,+,+,+,+
5,Ataxia,+,+,+,+
6,Gait ataxia,+,+,+,+
7,Dysarthria,+,+,+,+
8,Hyporeflexia,+,+,+,+
9,Impaired continence,-,-,-,+


Some column names might include spaces in front or after, and a couple of columns are subheadings and only contain NaNs, so lets correct that. Furthermore, remove individuals without an age specified or a variant in this gene.

In [5]:
df = df.set_index('Patient').T.reset_index()
df = df.dropna(axis=1, how='all')
df['patient_id'] = df["index"]
df = df[~df['Age'].isna()]
df = df[~df['Variant'].isna()]
df

Patient,index,Sex,Age,Seizures,Motor delay,Developmental delay,Ataxia,Gait ataxia,Dysarthria,Hyporeflexia,Impaired continence,Nystagmus,Variant,patient_id
0,II,female,19,+,+,+,+,+,+,+,-,+,c.139C>A,II
1,II2,female,18,+,+,+,+,+,+,+,-,+,c.139C>A,II2
2,II3,male,16,+,+,+,+,+,+,+,-,+,c.139C>A,II3
3,II4,female,10,+,+,+,+,+,+,+,+,+,c.139C>A,II4


<h2>Column mappers</h2>
<p>Please see the notebook "Create phenopackets from tabular data with individuals in rows" for explanations. In the following cell we create a dictionary for the ColumnMappers. Note that the code is identical except that we use the df.loc function to get the corresponding row data</p>

In [6]:
hpo_cr = parser.get_hpo_concept_recognizer()
column_mapper_d, col_not_found = try_mapping_columns(df=df,
                                                    observed='+',
                                                    excluded='-',
                                                    hpo_cr=hpo_cr,
                                                    preview=True)

                   term    status
0  Seizure (HP:0001250)  observed
1  Seizure (HP:0001250)  observed
2  Seizure (HP:0001250)  observed
3  Seizure (HP:0001250)  observed
                       term    status
0  Motor delay (HP:0001270)  observed
1  Motor delay (HP:0001270)  observed
2  Motor delay (HP:0001270)  observed
3  Motor delay (HP:0001270)  observed
                                      term    status
0  Global developmental delay (HP:0001263)  observed
1  Global developmental delay (HP:0001263)  observed
2  Global developmental delay (HP:0001263)  observed
3  Global developmental delay (HP:0001263)  observed
                  term    status
0  Ataxia (HP:0001251)  observed
1  Ataxia (HP:0001251)  observed
2  Ataxia (HP:0001251)  observed
3  Ataxia (HP:0001251)  observed
                       term    status
0  Gait ataxia (HP:0002066)  observed
1  Gait ataxia (HP:0002066)  observed
2  Gait ataxia (HP:0002066)  observed
3  Gait ataxia (HP:0002066)  observed
                    

In [7]:
print(col_not_found)

['index', 'Sex', 'Age', 'Variant', 'patient_id']


<h2>Variant Data</h2>
<p>The variant data (HGVS< transcript) is listed in the Variant (hg19, NM_015133.4) column.</p>

In [8]:
genome = 'hg19'
default_genotype = 'heterozygous'
transcript='NM_016373.2'
varMapper = VariantColumnMapper(assembly=genome,column_name='Variant', 
                                transcript=transcript)

<h1>Demographic data</h1>
<p>pyphetools can be used to capture information about age, sex, and individual identifiers. This information is stored in a map of "IndividualMapper" objects. Special treatment may be required for the indifiers, which may be used as the column names or row index.</p>

In [9]:
ageMapper = AgeColumnMapper.by_year('Age')
ageMapper.preview_column(df['Age'])

,original column contents,age
0,19,P19Y
1,18,P18Y
2,16,P16Y
3,10,P10Y


In [10]:
sexMapper = SexColumnMapper(male_symbol='male', female_symbol='female', column_name='Sex')
sexMapper.preview_column(df['Sex'])

,original column contents,sex
0,female,FEMALE
1,female,FEMALE
2,male,MALE
3,female,FEMALE


In [11]:
pmid = "PMID:17470496"
encoder = CohortEncoder(df=df, hpo_cr=hpo_cr, column_mapper_d=column_mapper_d, 
                        individual_column_name="patient_id", agemapper=ageMapper, sexmapper=sexMapper,
                       variant_mapper=varMapper, metadata=metadata,
                       pmid=pmid)
encoder.set_disease(disease_id='614322', label='Spinocerebellar ataxia, autosomal recessive 12')

In [12]:
individuals = encoder.get_individuals()

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_016373.2%3Ac.139C>A/NM_016373.2?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_016373.2%3Ac.139C>A/NM_016373.2?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_016373.2%3Ac.139C>A/NM_016373.2?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_016373.2%3Ac.139C>A/NM_016373.2?content-type=application%2Fjson


In [13]:
i1 = individuals[0]
phenopacket1 = i1.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh())
json_string = MessageToJson(phenopacket1)
print(json_string)

{
  "id": "II",
  "subject": {
    "id": "II",
    "timeAtLastEncounter": {
      "age": {
        "iso8601duration": "P19Y"
      }
    },
    "sex": "FEMALE"
  },
  "phenotypicFeatures": [
    {
      "type": {
        "id": "HP:0001250",
        "label": "Seizure"
      }
    },
    {
      "type": {
        "id": "HP:0001270",
        "label": "Motor delay"
      }
    },
    {
      "type": {
        "id": "HP:0001263",
        "label": "Global developmental delay"
      }
    },
    {
      "type": {
        "id": "HP:0001251",
        "label": "Ataxia"
      }
    },
    {
      "type": {
        "id": "HP:0002066",
        "label": "Gait ataxia"
      }
    },
    {
      "type": {
        "id": "HP:0001260",
        "label": "Dysarthria"
      }
    },
    {
      "type": {
        "id": "HP:0001265",
        "label": "Hyporeflexia"
      }
    },
    {
      "type": {
        "id": "HP:0031064",
        "label": "Impaired continence"
      },
      "excluded": true
    },
   

In [14]:
output_directory = "../../phenopackets/WWOX/"
encoder.output_phenopackets(outdir=output_directory)

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_016373.2%3Ac.139C>A/NM_016373.2?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_016373.2%3Ac.139C>A/NM_016373.2?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_016373.2%3Ac.139C>A/NM_016373.2?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_016373.2%3Ac.139C>A/NM_016373.2?content-type=application%2Fjson
Wrote 4 phenopackets to ../../phenopackets/WWOX/
